In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import random
import copy

import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.neighbors import kneighbors_graph

from louvain_communities.louvain import detect_communities, modularity

# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

seq_path = "mRNA_sequences.csv"
labels_path_minus = "A_minus_normalized_levels.csv"

In [3]:
def cnn_model(print_model_summary=False):
    """
    creates the tensorflow model
    """
    model = models.Sequential()
    model.add(layers.Conv1D(64, 10, activation='relu', input_shape=(110, 4)))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.mean_squared_error,
                  metrics=[tf.keras.losses.mean_squared_error])
    if print_model_summary:
        print(model.summary())
    return model


def one_hot_encoding(seq):
    """
    one hot encoding function - convet sequence of lentgh L into (L, 4) one-hot matrix
    """
    mapping = dict(zip("ACGT", range(4)))    
    seq2 = [mapping[i] for i in seq]

    return np.eye(4)[seq2].astype('uint8')


def load_data():
    # load the dataset and create the one-hot matrices
    seq = pd.read_csv(seq_path)["seq"].values.tolist()
    features_list = [one_hot_encoding(x) for x in seq]
    features = np.asarray(features_list, dtype="uint8")

    labels = pd.read_csv(labels_path_minus)
    labels = labels[["{}h".format(i) for i in [1,2,3,4,5,6,7,8,10]]].values

    # convert the mRNA levels into degradation rate
    deg_rates = []
    for i in range(len(labels)):
        reg = LinearRegression()
        reg.fit(np.array([1, 2, 3, 4, 5, 6, 7, 8, 10])[:, np.newaxis], labels[i, :][:, np.newaxis])
        deg_rates.append(reg.coef_)
        
    deg_rates = np.squeeze(np.array(deg_rates))

    return features, deg_rates

In [4]:
def kmer_link_graph(k=20, l=110, seq_file_path="mRNA_sequences.csv"):
    """
    creates a graph where seqences are nodes and link is formes if there is k-mer share

    k : k-mer length. Default: 20
    l : sequence fixed length. Default: 110
    seq_file_path: str
    """
    seq_list = pd.read_csv(seq_file_path)["seq"].values.tolist()
    hash_map = {}
    l = 110 # sequence fixed length
    G = nx.Graph()
    G.add_nodes_from(range(len(seq_list)))

    #passing all k-mer in the seqences and create the edges accordingly 
    for i,seq in enumerate(seq_list):
        for j in range(l-k+1):
            kmer=seq[j:j+k]
            if kmer in hash_map:
                hash_map[kmer].add(i)
                edges_list = [(i,m) for m in hash_map[kmer]]
                G.add_edges_from(edges_list)
            else:
                hash_map[kmer] = {i}

    return G


def kmer_link_graph_different_weights(ks=(15, 16, 17, 18, 19, 20),
                                      l=110,
                                      seq_file_path="mRNA_sequences.csv"):
    """
    creates a graph where seqences are nodes and link is formes if
    there is k-mer share of minium length ks[0].
    Form different weight for different length: ks[0]/ks[-1], ks[1]/ks[-1], ..., ks[-1]/ks[-1]=1
    ks : k-mer lengths. Default: (15, 16, 17, 18, 19, 20)
    l : sequence fixed length. Default: 110
    seq_file_path: str
    """
    k_max = max(ks)
    seq_list = pd.read_csv(seq_file_path)["seq"].values.tolist()
    G = nx.Graph()
    G.add_nodes_from(range(len(seq_list)))
    for k in ks:
        hash_map = {}
        #passing all k-mer in the seqences and create the edges accordingly 
        for i,seq in enumerate(seq_list):
            for j in range(l-k+1):
                kmer=seq[j:j+k]
                if kmer in hash_map:
                    hash_map[kmer].add(i)
                    edges_list = [(i,m) for m in hash_map[kmer]]
                    G.add_edges_from(edges_list, weight=k/k_max)
                else:
                    hash_map[kmer] = {i}

        return G

In [5]:
def divide_parition_to_train_and_test(partition, train_size=0.8, error_in_size=0.001):
    samples_size = sum(len(cluster) for cluster in partition)
    train_size = 0.8
    partition_temp = copy.deepcopy(partition)
    train_indices = []
    while len(train_indices) < (train_size - error_in_size) * samples_size:
        i = random.randint(0, len(partition_temp)-1)
        if len(train_indices) + len(partition_temp[i]) < (train_size + error_in_size) * samples_size:
            cluster_indices = partition_temp.pop(i)
            train_indices.extend(cluster_indices)
    test_indices = [index for cluster in partition_temp for index in cluster]
    print("train size:", len(train_indices))
    print("test size:", len(test_indices))

    return train_indices, test_indices

In [6]:
def extract_train_test_data_from_partition(features, deg_rates, train_indices, test_indices):
    train_x = features[train_indices]
    train_y = deg_rates[train_indices]
    test_x = features[test_indices]
    test_y = deg_rates[test_indices]

    return train_x, test_x, train_y, test_y


def create_train_and_evaluate_model(train_x, test_x, train_y, test_y, verbose=0):
    """
    returns the train model and the Pearson score on the test set
    """
    model = cnn_model()
    model.fit(train_x,
              train_y,
              #validation_data=(validation_x, validation_y),
              shuffle=True,
              epochs=10,                             
              batch_size= 64,
              verbose=verbose
              )
    test_y_predict = np.squeeze(model.predict(test_x))

    score = stats.pearsonr(test_y, test_y_predict)

    return model, score[0]

In [ ]:
def expriment_train_test(partition, random_train=False):
    features, deg_rates = load_data()
    louvain_partition_scores, random_partition_scores = [], []
    num_of_expriments, num_of_trains = 10, 10
    for i in range(num_of_expriments):
        print("louvain partition training:")
        train_indices, test_indices = divide_parition_to_train_and_test(partition, train_size=0.8)
        for j in range(num_of_trains):
            train_x, test_x, train_y, test_y = extract_train_test_data_from_partition(features, deg_rates, train_indices, test_indices)
            model, score = create_train_and_evaluate_model(train_x, test_x, train_y, test_y)
            louvain_partition_scores.append(score)
        if random_train:
            print("random partition training:")
            train_x, test_x, train_y, test_y = train_test_split(features, deg_rates, test_size=0.2, random_state=i)
            for j in range(num_of_trains):
                model, score = create_train_and_evaluate_model(train_x, test_x, train_y, test_y)
                random_partition_scores.append(score)
        print("finished expriment", i)

        louvain_partition_scores_arr = np.array(louvain_partition_scores)
        louvain_partition_scores_arr = louvain_partition_scores_arr[~np.isnan(louvain_partition_scores_arr)]
        print("Louvain partition training:")
        print("average Pearson:", np.mean(louvain_partition_scores_arr))
        if random_train:
            random_partition_scores_arr = np.array(random_partition_scores)
            random_partition_scores_arr = random_partition_scores_arr[~np.isnan(random_partition_scores_arr)]
            print("Random partition training:")
            print("average Pearson:", np.mean(random_partition_scores_arr))

Single k-mer length link

In [9]:
G = kmer_link_graph()
print(G.size(), "edges")
partition = detect_communities(G, randomized=True)
print("Modularity for best partition:", modularity(G, partition))

524752 edges


In [ ]:
expriment_train_test(partition, random_train=True)

Different_weights for different k-mer length links

In [ ]:
G = kmer_link_graph_different_weights()
print(G.size(), "edges")
partition = detect_communities(G, randomized=True)
print("Modularity for best partition:", modularity(G, partition))

In [ ]:
expriment_train_test(partition, random_train=False)

The graph based kmer counting

In [32]:
from itertools import product
###############kmer count######################
#kmer utilities functions 
def CountKmer(seq, k):
    kFreq = {}
    for i in range (0, len(seq)-k+1):
        kmer = seq [i:i+k]
        if kmer in kFreq:
            kFreq [kmer] += 1
        else:
            kFreq [kmer] = 1
    return kFreq

def retutnAllKmers(min_kmer_length, max_kmer_length):
    kmer_list = []
    for i in range (min_kmer_length, max_kmer_length+1):
        kmer_list = kmer_list + [''.join(c) for c in product('ACGT', repeat=i)]
    return kmer_list

def createFeturesVector(allKmers, seqkMerCounter):
    AllKmersSize = len(allKmers)
    KmerCounterArray = np.zeros((AllKmersSize,1))
    for i in range (0, AllKmersSize):
        if allKmers[i] in seqkMerCounter:
              KmerCounterArray[i] = seqkMerCounter[allKmers[i]]
    return KmerCounterArray

def createFeturesVectorsForAllSeq(allKmers, min_kmer_length, max_kmer_length, sequences):
    num_of_kmers = len(allKmers)
    num_of_sequences = len(sequences)
    FeturesVectorsOfAllSeq = np.zeros((num_of_sequences, num_of_kmers, 1), dtype='int8')
    for i in range(num_of_sequences): 
        seq = sequences [i]
        seqkMerCounter = {}
        for j in range (min_kmer_length, max_kmer_length+1):
            seqkMerCounter = {**seqkMerCounter, **CountKmer(seq, j)}
        FeturesVectorsOfAllSeq[i] = createFeturesVector(allKmers, seqkMerCounter)
    return FeturesVectorsOfAllSeq

In [33]:
# load the dataset and create the k-mer counting matrix
seq = pd.read_csv(seq_path)["seq"].values.tolist()
allKmers = retutnAllKmers(4, 7)
FeturesVectorsOfAllSeq = createFeturesVectorsForAllSeq(allKmers, 4, 7, seq)
FeturesVectorsOfAllSeq = np.squeeze(FeturesVectorsOfAllSeq)

In [36]:
# reduce matrix dimension
pca = PCA(n_components=100)
pca.fit(FeturesVectorsOfAllSeq)
FeturesVectorsOfAllSeq_pca = pca.transform(FeturesVectorsOfAllSeq)

# bulid the graph matrix using kneighbors_graph based on Euclidean distace
A = kneighbors_graph(FeturesVectorsOfAllSeq_pca, 10, mode="distance")
A[A.nonzero()] = 1 / (A[A.nonzero()] + 1) # convert to similarity 

PCA(n_components=100)

In [ ]:
G = nx.Graph(A)
print(G.size(), "edges")
partition = detect_communities(G, randomized=True)
print("Modularity for best partition:", modularity(G, partition))

In [ ]:
expriment_train_test(partition, random_train=False)